In [1]:
import os 
import sys
from dotenv import load_dotenv 
from pathlib import Path
import nest_asyncio
# Add project root to Python path
project_root = str(Path().absolute().parent)
sys.path.append(project_root)

from evoagentx.models import OpenAILLMConfig, OpenAILLM
from evoagentx.workflow import WorkFlowGraph, WorkFlow
from evoagentx.agents import AgentManager
from evoagentx.workflow import WorkFlowGenerator
from examples.pdf_test_prompt import PDF_AGENT_SUGGESTION, formulate_goal
from evoagentx.tools.mcp import MCPToolkit
from evoagentx.tools.file_tool import FileTool
from examples.goal_maker import making_goal

nest_asyncio.apply()
load_dotenv() # Loads environment variables from .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

/home/junhual1/miniconda3/envs/evo/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Job Recommendation Workflow with EvoAgentX

This notebook demonstrates how to use EvoAgentX framework to:
1. Process a PDF resume
2. Search for job opportunities
3. Generate personalized job recommendations

In [2]:
# Configuration settings
output_file = project_root +"/examples/output/jobs/output.md"
mcp_config_path = project_root +"/examples/output/jobs/mcp_jobs.config"
target_directory = project_root +"/examples/output/jobs/"
module_save_path = project_root +"/examples/output/jobs/jobs_demo_4o_jpt_mini.json"
pdf_path = project_root +"/examples/output/jobs/test_pdf.pdf"


## Initialize LLM and Tools

In [3]:
# LLM configuration
openai_config = OpenAILLMConfig(
    model="gpt-4o-mini", 
    openai_key=OPENAI_API_KEY, 
    stream=True, 
    output_response=True, 
    max_tokens=16000
)
# Initialize the language model
llm = OpenAILLM(config=openai_config)

# Define goal
goal = f"""Read and analyze the pdf resume at {pdf_path}, 
then find 5 real job opportunities to this client by search the website."""
goal = making_goal(openai_config, goal)

# Get tools
mcp_toolkit = MCPToolkit(config_path=mcp_config_path)
tools = mcp_toolkit.get_tools()
tools.append(FileTool())



Goal: "Analyze the PDF resume located at /home/junhual1/projects/EvoAgentX/examples/output/jobs/test_pdf.pdf for relevant skills, qualifications, and experience. Use this analysis to find 5 real job opportunities for the client on a job search website. Ensure to extract essential job details including job title, company name, location, job description, job requirements, salary, and the link to the job postings. The only initial input for this workflow is this goal, and please make use of relevant tools to retrieve the necessary data without generating any information."


2025-05-21 16:49:07.858 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.000 | Total tokens: 471 | Current cost: $0.000 | Current tokens: 471
2025-05-21 16:49:07.859 | INFO     | evoagentx.tools.mcp:_from_config:275 - Configured MCP server: hirebase (Hiring information requesting service, must provide search keywords) with command: uvx


Running event loop


2025-05-21 16:49:08.267 | INFO     | evoagentx.tools.mcp:__init__:234 - Successfully connected to MCP servers
2025-05-21 16:49:08.268 | INFO     | evoagentx.tools.mcp:get_tools:290 - Added 2 tools from MCP server


## Generate Workflow
Uncomment this section to generate a new workflow. Currently using a saved workflow.

In [4]:
# Workflow Creation
wf_generator = WorkFlowGenerator(llm=llm, tools=tools)
workflow_graph: WorkFlowGraph = wf_generator.generate_workflow(goal=goal, agent_suggestion=PDF_AGENT_SUGGESTION)
# # Display workflow
# workflow_graph.display()
# Save workflow 
workflow_graph.save_module(module_save_path)

2025-05-21 16:49:08.275 | INFO     | evoagentx.workflow.workflow_generator:generate_workflow:80 - Generating a workflow for: Goal: "Analyze the PDF resume located at /home/junhual1/projects/EvoAgentX/examples/output/jobs/test_pdf.pdf for relevant skills, qualifications, and experience. Use this analysis to find 5 real job opportunities for the client on a job search website. Ensure to extract essential job details including job title, company name, location, job description, job requirements, salary, and the link to the job postings. The only initial input for this workflow is this goal, and please make use of relevant tools to retrieve the necessary data without generating any information." ...


## Thought
The task involves two main phases: analyzing a PDF document to gather skills and qualifications and using this analysis to find relevant job opportunities online. Given the complexity of extracting information from a PDF and then searching jobs, I have broken down the work into distinct sub-tasks. Each sub-task is modular and focused on a specific area of the overall task, ensuring clarity and coherence. 

## Goal
Analyze the PDF resume located at /home/junhual1/projects/EvoAgentX/examples/output/jobs/test_pdf.pdf for relevant skills, qualifications, and experience, and subsequently find 5 real job opportunities for the client on a job search website, extracting essential job details from the postings.

## Plan
```json
{
    "sub_tasks": [
        {
            "name": "pdf_analysis",
            "description": "Analyze the PDF resume to extract relevant skills, qualifications, and experience.",
            "reason": "This sub-task is essential to gather information needed f

2025-05-21 16:49:18.931 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.001 | Total tokens: 2924 | Current cost: $0.001 | Current tokens: 2453
2025-05-21 16:49:18.936 | INFO     | evoagentx.workflow.workflow_generator:generate_workflow:83 - Successfully generate the following workflow:
Task Name: pdf_analysis
Description: Analyze the PDF resume to extract relevant skills, qualifications, and experience.
Inputs:
  - goal (string, required): The user's goal in textual format.
Outputs:
  - extracted_skills (string, required): A summary of the relevant skills and qualifications extracted from the PDF resume.

Task Name: job_search
Description: Search for real job opportunities using the extracted skills and qualifications.
Inputs:
  - goal (string, required): The user's goal in textual format.
  - extracted_skills (string, required): The relevant skills and qualifications derived from the PDF analysis.
Outputs:
  - job_listings (array, required): A list of job opp


## Thought
In analyzing the sub-task "pdf_analysis", the objective is to extract relevant skills, qualifications, and experience from a PDF resume based on the user's provided goal. The only input is a textual representation of the user's goal, which will guide the analysis process. 

Given the requirements, the existing tools do not include a dedicated agent for parsing PDF files. Therefore, I will need to generate a new agent, focusing on extracting the necessary information from the specified PDF file. This new agent will leverage the capabilities of reading and extracting contents from the provided PDF, possibly using the `read_file` tool, and then condense this information into a summary of skills. 

The output will be critical for the next stage of the workflow, where this information is used to find job opportunities. 

## Objective
The objective of the sub-task is to analyze a PDF resume to extract relevant skills, qualifications, and experience based on the user's goal. This 

2025-05-21 16:49:31.657 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.002 | Total tokens: 6492 | Current cost: $0.001 | Current tokens: 3568
2025-05-21 16:49:31.660 | INFO     | evoagentx.workflow.workflow_generator:generate_agents:120 - Generating agents for subtask: job_search



Generated agents: {
    "selected_agents": [],
    "generated_agents": [
        {
            "name": "resume_analysis_agent",
            "description": "This agent analyzes a PDF resume to extract relevant skills, qualifications, and experience based on the user's goal.",
            "inputs": [
                {
                    "name": "goal",
                    "type": "string",
                    "description": "The user's goal in textual format.",
                    "required": true
                }
            ],
            "outputs": [
                {
                    "name": "extracted_skills",
                    "type": "string",
                    "description": "A summary of the relevant skills and qualifications extracted from the PDF resume.",
                    "required": true
                }
            ],
            "prompt": "### Objective\nAnalyze the provided PDF resume to extract relevant skills, qualifications, and experience based on the us

2025-05-21 16:49:42.107 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.002 | Total tokens: 10163 | Current cost: $0.001 | Current tokens: 3671
2025-05-21 16:49:42.111 | INFO     | evoagentx.core.module:save_module:436 - Saving WorkFlowGraph to /home/junhual1/projects/EvoAgentX/examples/output/jobs/jobs_demo_4o_jpt_mini.json



Generated agents: {
    "selected_agents": [],
    "generated_agents": [
        {
            "name": "job_search_agent",
            "description": "This agent utilizes the user's goal and the extracted skills to search for relevant job opportunities, retrieving essential job details from job search platforms.",
            "inputs": [
                {
                    "name": "goal",
                    "type": "string",
                    "description": "The user's goal in textual format.",
                    "required": true
                },
                {
                    "name": "extracted_skills",
                    "type": "string",
                    "description": "The relevant skills and qualifications derived from the PDF analysis.",
                    "required": true
                }
            ],
            "outputs": [
                {
                    "name": "job_listings",
                    "type": "array",
                    "description

'/home/junhual1/projects/EvoAgentX/examples/output/jobs/jobs_demo_4o_jpt_mini.json'

## Load and Execute Workflow

In [5]:
# Load saved workflow 
workflow_graph: WorkFlowGraph = WorkFlowGraph.from_file(module_save_path)

# Set up agent manager
agent_manager = AgentManager(tools=tools)
agent_manager.add_agents_from_workflow(workflow_graph, llm_config=openai_config)

# Initialize and execute workflow
workflow = WorkFlow(graph=workflow_graph, agent_manager=agent_manager, llm=llm)
workflow.init_module()
output = workflow.execute()

2025-05-21 16:49:42.248 | INFO     | evoagentx.workflow.workflow:get_next_task:121 - Task Execution Trajectory: None. Scheduling next subtask ...
2025-05-21 16:49:42.249 | INFO     | evoagentx.workflow.workflow:get_next_task:123 - The next subtask to be executed is: pdf_analysis
2025-05-21 16:49:42.250 | INFO     | evoagentx.workflow.workflow:async_execute:83 - Executing subtask: pdf_analysis


Next action: Based on the tasks' execution results, the next action to be executed is the 'tool_calling' action of 'resume_analysis_agent' agent.
_______________________ Start Tool Calling _______________________
prompt_params_values:
{'goal': 'Goal: "Analyze the PDF resume located at /home/junhual1/projects/EvoAgentX/examples/output/jobs/test_pdf.pdf for relevant skills, qualifications, and experience. Use this analysis to find 5 real job opportunities for the client on a job search website. Ensure to extract essential job details including job title, company name, location, job description, job requirements, salary, and the link to the job postings. The only initial input for this workflow is this goal, and please make use of relevant tools to retrieve the necessary data without generating any information."'}
_______________________ Execution History _______________________
execution_history:
[]
{
    "function_params":[
        {
            "function_name": "read_file", 
          

2025-05-21 16:49:44.317 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.003 | Total tokens: 11903 | Current cost: $0.000 | Current tokens: 1740



Tool call args:
{
    "function_params": [
        {
            "function_name": "read_file",
            "function_args": {
                "file_path": "/home/junhual1/projects/EvoAgentX/examples/output/jobs/test_pdf.pdf"
            }
        }
    ],
    "continue_after_tool_call": false
}
_____________________ Start Function Calling _____________________
Executing function calling: read_file with {'file_path': '/home/junhual1/projects/EvoAgentX/examples/output/jobs/test_pdf.pdf'}
results:
{'result': [{'success': True, 'content': ' \n    \n   Wei Zhang \nEmail: zhang.wei@email.com | Phone: (555) 123-4567 | LinkedIn: linkedin.com/in/zhangwei \nGitHub: github.com/zhangwei-dev | Location: Beijing, China  \nEducation \nThe University of Melbourne , Victoria, Australia  Sept 2015 - June 2018  \n- Thesis: "Distributed Deep Learning Systems for Real-time Image Recognition" \n- Coursework: Advanced Algorithms, Machine Learning, Distributed Systems, Cloud Computing \n \nThe University of 

2025-05-21 16:49:47.406 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.003 | Total tokens: 13209 | Current cost: $0.000 | Current tokens: 1306
2025-05-21 16:49:47.409 | INFO     | evoagentx.workflow.workflow:get_next_task:121 - Task Execution Trajectory: pdf_analysis. Scheduling next subtask ...
2025-05-21 16:49:47.410 | INFO     | evoagentx.workflow.workflow:get_next_task:123 - The next subtask to be executed is: job_search
2025-05-21 16:49:47.411 | INFO     | evoagentx.workflow.workflow:async_execute:83 - Executing subtask: job_search


...

2025-05-21 16:49:55.165 | INFO     | evoagentx.tools.mcp:_sync_call_tool:203 - MCP tool search_jobs call completed successfully


results:
{'result': [CallToolResult(meta=None, content=[TextContent(type='text', text='{\n  "_id": "682cd69c8289a5ffad061368",\n  "company_name": "Kyndryl",\n  "job_title": "Lead, Customer Technology Advisor",\n  "description": "<p>We are looking for a Lead, Customer Technology Advisor to join our dynamic team as a member of our Go-To-Market team. As a certified expert in your field, you will be at the forefront of innovative technology solutions for our valued customers, collaborating with Kyndryl Consult Partners and Customer Partners to leverage your technical and solution expertise to drive the opportunities we seek and exceed customer expectations.</p><h2>Requirements</h2><ul><li>Deep knowledge of service offerings and technical solutions in practice</li><li>Proven experience in converting excellent technical knowledge into practical customer insights and solutions</li><li>Experience in consulting sales</li><li>Recognition as an expert in the technology and/or solution area, inclu

2025-05-21 16:50:19.300 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.004 | Total tokens: 20788 | Current cost: $0.001 | Current tokens: 5294
2025-05-21 16:50:19.306 | INFO     | evoagentx.workflow.workflow:async_execute:98 - Extracting WorkFlow Output ...



Extracted output:
{
    "job_listings": [
        {
            "job_title": "Lead, Customer Technology Advisor",
            "company_name": "Kyndryl",
            "location": "Tokyo, Japan",
            "description": "We are looking for a Lead, Customer Technology Advisor to join our dynamic team as a member of our Go-To-Market team. As a certified expert in your field, you will be at the forefront of innovative technology solutions for our valued customers, collaborating with Kyndryl Consult Partners and Customer Partners to leverage your technical and solution expertise to drive the opportunities we seek and exceed customer expectations.",
            "requirements": "5+ years of experience in technical services sales across multiple industries, expert in technology and/or solution area, and certified in areas such as application data and AI, cloud, core enterprise, and zCloud, digital workplace, networks and edge, security and resilience.",
            "salary": null,
          

2025-05-21 16:50:31.880 | INFO     | evoagentx.models.model_utils:update_cost:65 - Total cost: $0.005 | Total tokens: 23807 | Current cost: $0.001 | Current tokens: 3019


## Save and Display Results

In [7]:
# Save results to file
try:
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(output)
    print(f"Job recommendations have been saved to {output_file}")
except Exception as e:
    print(f"Error saving job recommendations: {e}")

# Display output
print(output)

Job recommendations have been saved to /home/junhual1/projects/EvoAgentX/examples/output/jobs/output.md
### Final Output:

```json
{
    "job_listings": [
        {
            "job_title": "Lead, Customer Technology Advisor",
            "company_name": "Kyndryl",
            "location": "Tokyo, Japan",
            "description": "We are looking for a Lead, Customer Technology Advisor to join our dynamic team as a member of our Go-To-Market team. As a certified expert in your field, you will be at the forefront of innovative technology solutions for our valued customers, collaborating with Kyndryl Consult Partners and Customer Partners to leverage your technical and solution expertise to drive the opportunities we seek and exceed customer expectations.",
            "requirements": "5+ years of experience in technical services sales across multiple industries, expert in technology and/or solution area, and certified in areas such as application data and AI, cloud, core enterprise, and